# Julia DEM SVD

In [1]:
import Pkg
Pkg.add("Flux")
Pkg.add("Plots")
Pkg.add("NetCDF")
Pkg.add("TSVD")
Pkg.add("IterTools")
Pkg.add("Statistics")
Pkg.add("Compat")
Pkg.add("LinearAlgebra")
Pkg.add("Glob")
using Flux
using Flux: train!
using Plots
using TSVD
using IterTools
using IterTools: ncycle 
using Statistics
using LinearAlgebra
using Compat
using Glob
using NetCDF

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed Libiconv_jll ─ v1.16.1+2
   Installed Glib_jll ───── v2.74.0+2
   Installed Qt5Base_jll ── v5.15.3+2
  No Changes to `~/.julia/environments/v1.8/Project.toml`
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [7746bdde] ↑ Glib_jll v2.74.0+1 ⇒ v2.74.0+2
  [94ce4f54] ↑ Libiconv_jll v1.16.1+1 ⇒ v1.16.1+2
  [ea2cea3b] ↑ Qt5Base_jll v5.15.3+1 ⇒ v5.15.3+2
Precompiling project...
  ✓ Libiconv_jll
  ✓ XML2_jll
  ✓ Wayland_jll
  ✓ XSLT_jll
  ✓ NetCDF_jll
  ✓ Gettext_jll
  ✓ Xorg_libxcb_jll
  ✓ Glib_jll
  ✓ Xorg_libX11_jll
  ✓ Xorg_xcb_util_jll
  ✓ Xorg_libXext_jll
  ✓ Xorg_libXfixes_jll
  ✓ Xorg_libXrender_jll
  ✓ Xorg_libxkbfile_jll
  ✓ Xorg_xcb_util_image_jll
  ✓ Xorg_xcb_util_renderutil_jll
  ✓ Xorg_xcb_util_keysyms_jll
  ✓ Xorg_xcb_util_wm_jll
  ✓ NetCDF
  ✓ Libglvnd_jll
  ✓ Xorg_libXinerama_jll
  ✓ Xorg_libXi_jll
  ✓ Xorg_libXrandr_jll
  ✓ Xorg_xkbcomp_jll
  ✓ Xorg_libXcursor_j

In [4]:
training_files = glob("usurf_ex_gris_g1800m_v5_RAGIS_id_*_1980-1-1_1990-1-1.nc")

nf = length(training_files)
d = ncread(training_files[1], "usurf")
nx, ny, nt = size(d)
x = ncread(training_files[1], "x")
y = ncread(training_files[1], "y")

Data = zeros(nf * nt, nx*ny)
for (k, training_file) in enumerate(training_files)
    println(training_file)
    d = ncread(training_file, "usurf")
    data = transpose(reshape(d, ny * nx, nt))
    Data[(k - 1 ) * nt + 1:k * nt, :] = data
end


obs_file ="aerodem_1978_1987_wgs84_g1800m.nc"
obs = ncread(obs_file, "surface_altitude")
nx_obs, ny_obs = size(obs)

R = reshape(obs, 1, nx * ny);
I = findall(x->x!=-9999, R);

LoadError: BoundsError: attempt to access 0-element Vector{String} at index [1]

In [3]:
q = 10

Data_mean = mean(Data, dims=1)
Data_scaled = Data .- Data_mean
U, S, V = tsvd(Data_scaled, q)
println("dim V = ", size(V))
println("dim S = ", size(S))
println("dim U = ", size(U))
println("dim R = ", size(R))

SV = transpose(V * diagm(S))

println("SV= ",size(SV))

x_train = transpose(SV[repeat(I, 1, q)])
_, n_obs = size(x_train)
y_train = reshape(R[I], 1, n_obs)

LoadError: UndefVarError: Data not defined

In [19]:
n_epochs = 100

for K in [0 1e4 1e5 1e6 1e7]
    println(K)
    model = Dense(q, 1)
    ps = Flux.params(model)
    penalty() = K * mean(abs2, model.weight)
    loss(x, y) = Flux.Losses.mse(model(x), y) + penalty()
    opt = Adam(0.1)

    loss_0 = loss(x_train, y_train)
    println("Initial loss: $loss_0")

    data = [(x_train, y_train)]

    train_loader = Flux.DataLoader((x_train, y_train), batchsize = 128, shuffle = true)
    Flux.train!(loss, ps, ncycle(train_loader, n_epochs), opt)

    loss_e = loss(x_train, y_train)
    println("Final loss: $loss_e")

    M =  reshape(model.weight * SV + Data_mean, nx, ny)
    M[findall(x->x<0, M)] .= 0

    Ms = copy(reshape(M, nx, ny, 1))

    filename = "dem_ragis_q_$(q)_K_$(K).nc"
    varname  = "usurf"
    attribs  = Dict("units"   => "m",
                    "data_min" => 0.0)

    nccreate(filename, varname, "x", x, "y", y, "time", 1, atts=attribs)
    ncwrite(Ms, filename, varname)
end

0.0
Initial loss: 605223.3035895247
Final loss: 271742.13113388815
10000.0
Initial loss: 552618.1580774006
Final loss: 274975.4951375171
100000.0
Initial loss: 545330.4120321254
Final loss: 271452.43984731
1.0e6
Initial loss: 673385.0616154557
Final loss: 273487.98514266516
1.0e7
Initial loss: 2.704817590303704e6
Final loss: 274854.53437216865


In [21]:
heatmap(x, y, transpose(M))



LoadError: UndefVarError: M not defined